# Vacuum Cleaner notebook

### Import needed modules

1. `from agents import *`: This line imports all classes and functions from the `agents` module. This module might contain definitions for various types of agents.

2. `from itertools import combinations`: This line imports the `combinations` function from the `itertools` module. The `combinations` function is used to generate all possible combinations of a certain number of elements.

3. `from random import choice`: This line imports the `choice` function from the `random` module. The `choice` function is used to select a random element from a list or sequence.

4. `from typing import List`: This line imports the `List` class from the `typing` module. `List` is a generic class used to annotate list types.

5. `import collections`: This line imports the `collections` module, which provides alternatives to some of Python's built-in data structures.

6. `import numpy as np`: This line imports the `numpy` module with the alias `np`. Numpy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.

7. `import sys`: This line imports the `sys` module, which provides access to some variables used or maintained by the Python interpreter and to functions that interact strongly with the interpreter.

8. At the end we check for Python version 3.10 or later versions. In these versions, some modules have been moved to the collections.abc package, so this block updates references to these modules.

In [1]:
from agents import *
from itertools import combinations
from random import choice
from typing import List
import collections
import numpy as np
import sys

# Only if needed for new python versions
if sys.version_info >= (3, 10):
    import collections
    collections.Iterable = collections.abc.Iterable
    collections.Sequence = collections.abc.Sequence

# Easy environment 

### In the following code we define an agent

---

1. `class SimpleRoomba(Agent):` This line defines a new class named `SimpleRoomba` that inherits from the `Agent` module.

2. `def moveforward(self, success=True):` This method moves the `SimpleRoomba` instance forward if the `success` parameter is `True`. The direction of movement is determined by the current direction of the instance.

3. `def turn(self, d):` This method changes the direction of the `SimpleRoomba` instance based on the value of `d`.

3. `def suck(self, thing):` This method returns `True` if the `thing` parameter is an instance of the `Dirt` class, and `False` otherwise.

---

#### Define the program of the simple reflex agent

`def default_program(percepts):` Defines a function that returns an action based on its percepts. It iterates over each percept in `percepts`, and if a percept is an instance of the `Dirt` class, it returns 'suck'. If a percept is an instance of the `Bump` class, it randomly chooses an action from 'turnright' and 'turnleft'. Otherwise, it randomly chooses an action from 'turnright', 'turnleft', and 'moveforward'.

In [2]:
class SimpleRoomba(Agent):
    location = [0,0]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def suck(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Dirt):
            return True
        return False
        
def default_program(percepts):
    '''Returns an action based on it's percepts'''
    for p in percepts:
        if isinstance(p, Dirt):
            return 'suck'
        # Check if you are at an edge and have to turn
        motions = ['turnright', 'turnleft', 'moveforward']
        
        if isinstance(p, Bump): 
            choice = random.choice(motions[:-1]); # Do not move forward
        else:
            choice = random.choice(motions)
            
    return choice

### Define a simple enviroment
> as showed during the turoring

This enviroment will be used to get performance metric averaged over all the enviroments

In [3]:
class VacumEnv(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            agent.turn(Direction.R)
            agent.performance -= 1
        elif action == 'turnleft':
            agent.turn(Direction.L)
            agent.performance -= 1
        elif action == 'moveforward':
            agent.moveforward()
            agent.performance -= 1
        elif action == "suck":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.suck(items[0]):
                    self.delete_thing(items[0])
                    agent.performance += 100
                    
    def is_done(self):
        '''By default, we're done when there is no dirt left'''
        no_dirt = not any(isinstance(thing, Dirt) for thing in self.things)
        if no_dirt == True:
            print("Everything clean")
        return no_dirt
    
    # Some small overrides to pre existing functions
    # ________________________________________________
    
    def run(self, steps=100, delay=0, draw=False):
        for step in range(steps):
            self.update(delay, draw)
            if self.is_done():
                break
            self.step()
        self.update(delay,draw)
        
    def update(self, delay=0, draw=False):
        sleep(delay)
        self.reveal(draw)

    def reveal(self, draw=False):
        self.draw_world()
        # apply changes to the same grid instead
        # of making a new one.
        if draw==True:
            clear_output(1) # This shows the enviroment
            self.grid.show()
        
    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

#### Define `run_simulation(AgentFactory, width, height, num_steps)`
> Generate all possible dirt configurations

And the get the average of the agent performances

In [4]:
def run_simulation(AgentFactory, width, height, num_steps):

    dirt_configs = [[1 if j in combo else 0 for j in range(width * height)] for i in range(1, width * height + 1)\
                    for combo in combinations(range(width * height), i)]

    total_performance = 0

    # Run the simulation for each dirt configuration
    for config in dirt_configs:
        env = VacumEnv(width, height, color={'SimpleRoomba': (200, 0, 0), 'Dirt': (125, 100, 80)})
        agent = AgentFactory()
        env.add_thing(agent, [0, 0]) # Adding the agent
        for index, value in enumerate(config):
            if value == 1:
                dirt = Dirt()
                x = index % width
                y = index // height
                env.add_thing(dirt, [x, y]) # Adding the dirt
        env.run(num_steps, 0, False)
        total_performance += agent.performance

    average_performance = total_performance / len(dirt_configs)
    print("Average performance:", average_performance)

In [5]:
run_simulation(lambda: SimpleRoomba(default_program), 2, 2, num_steps=100)

Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Everything clean
Average performance: 160.33333333333334


# More Complex environment 

### Define our Agenet Class
> The agent is a vacum cleaner

Has the following capabilities:

- MoveRight
- MoveDown
- MoveLeft
- Suck

In [6]:
class Roomba(Agent):
    def MoveRight(self) -> None:
        '''MoveRight possible only if success (i.e. valid destination location)'''
        if self.collided[0]:
            return
        self.location = (self.location[0] + 1, self.location[1])
        
    def MoveDown(self) -> None:
        '''MoveDown possible only if success (i.e. valid destination location)'''
        if self.collided[1]:
            return
        self.location = (self.location[0], self.location[1] + 1)
            
    def MoveLeft(self) -> None:
        '''MoveLeft possible only if success (i.e. valid destination location)'''
        if self.collided[2]:
            return
        self.location = (self.location[0] - 1, self.location[1])
        
    def MoveUp(self) -> None:
        '''MoveUp possible only if success (i.e. valid destination location)'''
        if self.collided[3]:
            return
        self.location = (self.location[0], self.location[1] - 1)
        
    def Suck(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Dirt):
            return True
        return False

### Enviroment
> Defining a Vacumm Cleaner environment

Our `Roomba_Env` inherits from the class `XYEnvironment` and exteds the class with a few more functionalities to make the word a bit more complex

In [7]:
class Roomba_Env(XYEnvironment):
    # Define members of the class Roomba_Env
    # ______________________________________
    
    # Probability to spawn a pit in a location
    dirt_probability = random.uniform(0.1, 0.3)
    # Probability to spawn an obstacle in a location
    obstacle_probability = random.uniform(0.1, 0.9) 
    
    # Define the global variable for the colors
    colors = {
        #'Obstacle': (80, 80, 80),
        'Obstacle': (44, 53, 57),
        'Wall': (44, 53, 57),
        'Roomba': (200, 0, 0),
        'Dirt': (125, 100, 80)}
    #________________________________________
    
    # Room should be random size + 3
    def __init__(self, verbose = False, random = True, width=9, height=9):
        if random == True:
            # Generate random dimensions between 9 and 15
            width = np.random.randint(9, 15)
            height = np.random.randint(9, 15)
        
        self.verbose = verbose
        
        # Call the above class with the custom size
        super().__init__(width, height)
        
        # Creates a grid
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        self.bounded = True
        
        self.init_world()
           
    def add_walls(self) -> None:
        """Put walls around the entire perimeter of the grid."""
        num_layers = 2
        for layer in range(num_layers):
            for x in range(layer, self.width - layer):
                if layer == 0 or random.random() < (num_layers - layer) / num_layers:
                    self.add_thing(Wall(), (x, layer))
                if layer == 0 or random.random() < (num_layers - layer) / num_layers:
                    self.add_thing(Wall(), (x, self.height - 1 - layer))
                    
            for y in range(layer + 1, self.height - 1 - layer):
                if layer == 0 or random.random() < (num_layers - layer) / num_layers:
                    self.add_thing(Wall(), (layer, y))
                if layer == 0 or random.random() < (num_layers - layer) / num_layers:
                    self.add_thing(Wall(), (self.width - 1 - layer, y))
                    
        # Check corners of the second layer
        corners = [(1, 1), (1, self.height - 2), (self.width - 2, 1), (self.width - 2, self.height - 2)]

        # Check each corner
        for x, y in corners:
            # Add a wall if there are 4 things
            if sum(1 for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]
                        for _ in self.list_things_at((x + dx, y + dy))) == 4:
            
                self.add_thing(Wall(), (x, y), True)
 
                    
    def add_obstacles(self, x_bounds: List[int], y_bounds: List[int]) -> None:
        """Add objects inside grid that work like walls."""
        # first 4 all essentialy because we do not want chess grid like walls 
        # The second 2 are becuase we do not want full lines of walls + obstacles which block the agent
        surroundings = [(1 , 1), (-1, -1), (-1, 1), (1, -1), (1, 0), (0, 1)]
        
        # Loops to add the obstacles
        for x in range(*x_bounds):
            for y in range(*y_bounds):
                if random.random() < self.obstacle_probability:
                    # Rivedi  
                    # Check if the location is not a wall and has at least one free adjacent cell in every direction of movement
                    if all(self.is_inbounds((x + dx, y + dy)) and \
                        not any((isinstance(thing, Wall) or isinstance(thing, Obstacle)) for thing in self.list_things_at((x + dx, y + dy))) for dx, dy in surroundings):
                        self.add_thing(Obstacle(), (x, y), True)
                        
    def add_dirt(self, x_bounds: List[int], y_bounds: List[int]) -> None:
        """Add dirt inside grid"""
        for x in range(*x_bounds):
            for y in range(*y_bounds):
                if random.random() < self.dirt_probability and \
                not any((isinstance(thing, Obstacle) or (isinstance(thing, Wall))) for thing in self.list_things_at((x, y))):
                    # Then you can add the dirt
                    self.add_thing(Dirt(), (x, y), True)
                    
    def init_world(self,) -> None:
        """Spawn items in the world based on probabilities from the book"""
        # Bounds for the objects and dirt to not overlap with the walls
        x_bounds = [1, self.x_end - 1]
        y_bounds = [1, self.y_end - 1]
    
        # add walls objects and dit 
        self.add_walls()
        self.add_obstacles(x_bounds, y_bounds)
        self.add_dirt(x_bounds, y_bounds)

        
    def get_world(self) -> List[List[object]]:
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width, self.height

        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
            
        return result
    
    def percepts_from(self, agent, location, tclass=Thing):
        """Return percepts from a given location"""
        thing_percepts = {
            Obstacle: Bump(),
            Wall: Bump(),
        }
        # Create the results of what is in that position
        result = [thing_percepts.get(thing.__class__, thing) for thing in self.list_things_at(location)]
        
        return result if len(result) else [None]

    def percept(self, agent):
        """Return things in adjacent (not diagonal) cells of the agent.
        Result format: [Left, Right, Up, Down, Center / Current location]"""
        x, y = agent.location
        result = []
        # Clockwise movements remember that the grid you seed as y fliipped
        result.append(self.percepts_from(agent, (x + 1, y))) # Right
        result.append(self.percepts_from(agent, (x, y + 1))) # Down: really important
        result.append(self.percepts_from(agent, (x - 1, y))) # Left 
        result.append(self.percepts_from(agent, (x, y - 1))) # Up: be careful (-1) is up
        result.append(self.percepts_from(agent, (x, y))) # In that position
        
        # List of percepts
        return result
    
    def execute_action(self, agent, action) -> None:
        '''changes the state of the environment based on what the agent does.'''
        agent.collided = 4 * [False]
        
        # Get the percepts that I care about
        percepts = self.percept(agent)[:-1]
        
        # Create an array with True if obstacles are present in that direction and False if not
        obstacle = [isinstance(p[0], Bump) for p in percepts]
        
        # Directions are clockwise from 0 -> move right, 1 -> move down ...
        movement  = {
            'MoveRight': agent.MoveRight,
            'MoveDown': agent.MoveDown,
            'MoveLeft': agent.MoveLeft,
            'MoveUp': agent.MoveUp,
        }
        
        # Check for all possible movments
        if action in movement:
            if self.verbose:
                print(f'{str(agent)[1:-1]} decided to {action} at location: {agent.location}')
            
            # Get the index
            index = list(movement.keys()).index(action)
           
            # if the agents bumps set collision in that direction to true
            if obstacle[index] == True:
                agent.collided[index] = True

            # Do your action
            movement[action]()
            agent.performance -= 1
            
        # Else check for the action of sucking Dirt
        elif action == "Suck":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if items: # which is the same as asking if len(items) != 0:
                if agent.Suck(items[0]):
                    if self.verbose:
                        print(f'{str(agent)[1:-1]} suck {str(items[0])[1:-1]} at location: {agent.location}, :)')
                    self.delete_thing(items[0])
                    agent.performance += 100
                    
    def is_done(self):
        no_dirt = False # True when there is no dirt left
        if not any(isinstance(thing, Dirt) for thing in self.things):
            if self.verbose:
                print("There is not dirt left")
            no_dirt = True 
        return no_dirt # Return if the game is ended
    
    # Some small overrides to pre existing functions
    # ________________________________________________
    
    def run(self, steps=100, delay=0, draw=False):
        for step in range(steps):
            self.update(delay, draw)
            if self.is_done():
                break
            self.step()
        self.update(delay,draw)
        
    def update(self, delay=0, draw=False):
        sleep(delay)
        self.reveal(draw)

    def reveal(self, draw=False):
        self.draw_world()
        # apply changes to the same grid instead
        # of making a new one.
        if draw==True:
            clear_output(1) # This shows the enviroment
            self.grid.show()
        
    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

#### Define the program for the simple reflex agent
> The simple reflex agent will happly bump against a wall as many times as he wants

In [8]:
def simple_program(percepts):
    '''Returns an action based on it's percepts'''
    # If you are on top of dirt Suck since percepts[-1][0] is what the agent is on top on
    # and percepts[-1][1] is agent itselfe
    if isinstance(percepts[-1][0], Dirt):
        return 'Suck'
   
    # Chooses a random direction
    directions = ['MoveRight', 'MoveDown', 'MoveLeft', 'MoveUp']
    return random.choice(directions)

##### Function to place the agent in a random location

In [9]:
# Define a function to get a random position that is not on top of walls, dirt, or obstacles
def get_random_position(room):
    while True:
        x = random.randint(1, room.width - 2)
        y = random.randint(1, room.height - 2)
        # Good way to write the if statement
        objects = [Wall, Dirt, Obstacle]
        if not any(isinstance(obj, var) for var in objects for obj in room.list_things_at((x, y))):
            return (x, y)

### Finally run the enviroment for tot steps

In [10]:
# Create the enviroment with the correct program
room = Roomba_Env(verbose=True)

# Add the Roomba object to the environment at a random position
room.add_thing(Roomba(simple_program), get_random_position(room), True)

# Run the rumba for x step with 0 delay because we want to be fast but still show the cool drawings
room.run(steps = 10, delay = 0, draw=True)

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Roomba decided to MoveDown at location: (4, 6)


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Roomba decided to MoveDown at location: (4, 7)


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


...

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Roomba decided to MoveDown at location: (5, 9)


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Roomba decided to MoveRight at location: (5, 9)


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


### Define the program for an agent which avoids the walls
> The simple reflex agent will happly bump against a wall as many times as he wants

In [11]:
def wall_avoid_program(percepts):
    '''returns an action based on it's percepts'''
    # if you are on top of dirt suck since percepts[-1][0] is what the agent is on top on
    # and percepts[-1][1] is agent itselfe
    if isinstance(percepts[-1][0], Dirt):
        return 'Suck'
        
    directions = {0: 'MoveRight', 1: 'MoveDown', 2: 'MoveLeft', 3: 'MoveUp'}

    # Check for percepts near you
    for i, p in enumerate(percepts[:-1]):
        # then check if it bumps against something 
        if isinstance(p[0], Bump):
            # remove the direction corresponding to the index of the if case
            del directions[i]
            
    # Check if directions is empty
    if not directions:
        raise Exception("No available directions")
    
    # Chooses a random direction
    return random.choice(list(directions.values()))

In [12]:
# Create the enviroment with the correct program
room = Roomba_Env(verbose=True)

# Add the Roomba object to the environment at a random position
room.add_thing(Roomba(wall_avoid_program), get_random_position(room), True)

# Run the rumba for x step with 0 delay because we want to be fast but still show the cool drawings
room.run(steps = 10, delay = 0, draw=True)

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Roomba decided to MoveDown at location: (2, 2)


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Roomba decided to MoveDown at location: (2, 3)


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


...

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Roomba decided to MoveRight at location: (3, 6)


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Roomba decided to MoveRight at location: (4, 6)


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


### Defining the test function 
> The definition can be found inside agents.py

In [13]:
# Import for progress bar
from tqdm import tqdm

# Custom implementation to allows the agent to be placed in a random position
def test_agent(AgentFactory, steps, envs):
    def compute_score(env):
        agent = AgentFactory()
        env.add_thing(agent, get_random_position(room), True) # maybe
        # env.add_thing(agent) # maybe it already places in a correct location questionable
        env.run(steps)
        return agent.performance
    
    def mean_score_with_progress(envs):
        scores = []
        for env in tqdm(envs):
            score = compute_score(env)
            scores.append(score)

        return mean(scores)

    return mean_score_with_progress(envs)

#### Testing in parallel: 

- ##### Simple Reflex agent
- ##### Wall avoiding simple agent

In [14]:
import threading

def run_test_agent(program, steps, envs):
    print(f'The {program} achived: {test_agent(lambda: Roomba(program), steps, envs)}')

if __name__ == '__main__':
    n_envs = 500
    steps = 500
    envs = [Roomba_Env() for _ in range(n_envs)]
    t1 = threading.Thread(target=run_test_agent, args=(simple_program, steps, envs))
    t2 = threading.Thread(target=run_test_agent, args=(wall_avoid_program, steps, envs))
    t1.start()
    t2.start()
    t1.join()
    t2.join()


 19%|██████████▉                                                | 93/500 [05:28<31:42,  4.67s/it]

list.remove(x): x not in list
  in Environment delete_thing
  Thing to be removed: <Dirt> at (10, 4)
  from list: [(<Wall>, (0, 0)), (<Wall>, (0, 11)), (<Wall>, (1, 0)), (<Wall>, (1, 11)), (<Wall>, (2, 0)), (<Wall>, (2, 11)), (<Wall>, (3, 0)), (<Wall>, (3, 11)), (<Wall>, (4, 0)), (<Wall>, (4, 11)), (<Wall>, (5, 0)), (<Wall>, (5, 11)), (<Wall>, (6, 0)), (<Wall>, (6, 11)), (<Wall>, (7, 0)), (<Wall>, (7, 11)), (<Wall>, (8, 0)), (<Wall>, (8, 11)), (<Wall>, (9, 0)), (<Wall>, (9, 11)), (<Wall>, (10, 0)), (<Wall>, (10, 11)), (<Wall>, (11, 0)), (<Wall>, (11, 11)), (<Wall>, (12, 0)), (<Wall>, (12, 11)), (<Wall>, (13, 0)), (<Wall>, (13, 11)), (<Wall>, (0, 1)), (<Wall>, (13, 1)), (<Wall>, (0, 2)), (<Wall>, (13, 2)), (<Wall>, (0, 3)), (<Wall>, (13, 3)), (<Wall>, (0, 4)), (<Wall>, (13, 4)), (<Wall>, (0, 5)), (<Wall>, (13, 5)), (<Wall>, (0, 6)), (<Wall>, (13, 6)), (<Wall>, (0, 7)), (<Wall>, (13, 7)), (<Wall>, (0, 8)), (<Wall>, (13, 8)), (<Wall>, (0, 9)), (<Wall>, (13, 9)), (<Wall>, (0, 10)), (<Wall>


 34%|███████████████████▊                                      | 171/500 [10:13<16:29,  3.01s/it]

list.remove(x): x not in list
  in Environment delete_thing
  Thing to be removed: <Dirt> at (9, 1)
  from list: [(<Wall>, (0, 0)), (<Wall>, (0, 9)), (<Wall>, (1, 0)), (<Wall>, (1, 9)), (<Wall>, (2, 0)), (<Wall>, (2, 9)), (<Wall>, (3, 0)), (<Wall>, (3, 9)), (<Wall>, (4, 0)), (<Wall>, (4, 9)), (<Wall>, (5, 0)), (<Wall>, (5, 9)), (<Wall>, (6, 0)), (<Wall>, (6, 9)), (<Wall>, (7, 0)), (<Wall>, (7, 9)), (<Wall>, (8, 0)), (<Wall>, (8, 9)), (<Wall>, (9, 0)), (<Wall>, (9, 9)), (<Wall>, (10, 0)), (<Wall>, (10, 9)), (<Wall>, (11, 0)), (<Wall>, (11, 9)), (<Wall>, (0, 1)), (<Wall>, (11, 1)), (<Wall>, (0, 2)), (<Wall>, (11, 2)), (<Wall>, (0, 3)), (<Wall>, (11, 3)), (<Wall>, (0, 4)), (<Wall>, (11, 4)), (<Wall>, (0, 5)), (<Wall>, (11, 5)), (<Wall>, (0, 6)), (<Wall>, (11, 6)), (<Wall>, (0, 7)), (<Wall>, (11, 7)), (<Wall>, (0, 8)), (<Wall>, (11, 8)), (<Wall>, (1, 1)), (<Wall>, (1, 8)), (<Wall>, (4, 1)), (<Wall>, (4, 8)), (<Wall>, (5, 8)), (<Wall>, (7, 8)), (<Wall>, (8, 1)), (<Wall>, (10, 1)), (<Wall>, 


 54%|███████████████████████████████▌                          | 272/500 [15:59<12:19,  3.24s/it]

list.remove(x): x not in list
  in Environment delete_thing
  Thing to be removed: <Dirt> at (1, 5)
  from list: [(<Wall>, (0, 0)), (<Wall>, (0, 13)), (<Wall>, (1, 0)), (<Wall>, (1, 13)), (<Wall>, (2, 0)), (<Wall>, (2, 13)), (<Wall>, (3, 0)), (<Wall>, (3, 13)), (<Wall>, (4, 0)), (<Wall>, (4, 13)), (<Wall>, (5, 0)), (<Wall>, (5, 13)), (<Wall>, (6, 0)), (<Wall>, (6, 13)), (<Wall>, (7, 0)), (<Wall>, (7, 13)), (<Wall>, (8, 0)), (<Wall>, (8, 13)), (<Wall>, (9, 0)), (<Wall>, (9, 13)), (<Wall>, (10, 0)), (<Wall>, (10, 13)), (<Wall>, (0, 1)), (<Wall>, (10, 1)), (<Wall>, (0, 2)), (<Wall>, (10, 2)), (<Wall>, (0, 3)), (<Wall>, (10, 3)), (<Wall>, (0, 4)), (<Wall>, (10, 4)), (<Wall>, (0, 5)), (<Wall>, (10, 5)), (<Wall>, (0, 6)), (<Wall>, (10, 6)), (<Wall>, (0, 7)), (<Wall>, (10, 7)), (<Wall>, (0, 8)), (<Wall>, (10, 8)), (<Wall>, (0, 9)), (<Wall>, (10, 9)), (<Wall>, (0, 10)), (<Wall>, (10, 10)), (<Wall>, (0, 11)), (<Wall>, (10, 11)), (<Wall>, (0, 12)), (<Wall>, (10, 12)), (<Wall>, (1, 1)), (<Wall>, 


100%|██████████████████████████████████████████████████████████| 500/500 [29:07<00:00,  3.49s/it]

 99%|█████████████████████████████████████████████████████████▌| 496/500 [29:09<00:16,  4.00s/it]

The <function simple_program at 0x1078a3ec0> achived: 441.522



100%|██████████████████████████████████████████████████████████| 500/500 [29:17<00:00,  3.52s/it]

The <function wall_avoid_program at 0x1078e5080> achived: 314.912
